In [1]:
#default_exp spec_lib.predict_lib

For most of the users, `alphadeep.pretrained_models.ModelManager`, and `AlphaDeep.spec_lib.predict_lib.PredictSpecLib`, and `AlphaDeep.protein.fasta.PredictFastaSpecLib` can cover most of the use cases.

Both  `AlphaDeep.spec_lib.predict_lib.PredictSpecLib` and `AlphaDeep.protein.fasta.PredictFastaSpecLib` take `alphadeep.pretrained_models.ModelManager` as the first positional arguments. `alphadeep.pretrained_models.ModelManager` must be initialized (`models.load_installed_models()`) before creating `...SpecLib` objects.

In [2]:
#export
import pandas as pd
import numpy as np

from alphabase.spectrum_library.library_base import SpecLibBase
from alphadeep.pretrained_models import ModelManager

class PredictSpecLib(SpecLibBase):
    def __init__(self,
        model_manager: ModelManager,
        charged_frag_types = ['b_z1','b_z2','y_z1','y_z2'],#['b_modloss_z1', ...]
        min_frag_mz = 50, max_frag_mz = 2000,
        min_precursor_mz = 400, max_precursor_mz = 2000,
    ):
        super().__init__(
            charged_frag_types,
            min_frag_mz=min_frag_mz,
            max_frag_mz=max_frag_mz,
            min_precursor_mz=min_precursor_mz,
            max_precursor_mz=max_precursor_mz
        )
        self.model_manager = model_manager

        self.intensity_factor = 1
        self.verbose = True

        self._precursor_df = pd.DataFrame()
        self._fragment_intensity_df = pd.DataFrame()
        self._fragment_mz_df = pd.DataFrame()

    @property
    def precursor_df(self):
        return self._precursor_df

    @precursor_df.setter
    def precursor_df(self, df):
        self._precursor_df = df
        self.refine_df()

    def sort_by_nAA(self):
        if 'nAA' not in self._precursor_df.columns:
            self._precursor_df[
                'nAA'
            ] = self._precursor_df.sequence.str.len().astype(np.int32)
        self._precursor_df.sort_values('nAA', inplace=True)
        self._precursor_df.reset_index(drop=True, inplace=True)

    def refine_df(self):
        """
        To make sure all columns have desired dtype.
        This function also sorts `nAA`, and `reset_index` for fast prediction.
        """
        if self._precursor_df.charge.dtype not in ['int','int8','int64','int32']:
            self._precursor_df['charge'] = self._precursor_df['charge'].astype(int)

        if self._precursor_df.mod_sites.dtype not in ['O','U']:
            self._precursor_df['mod_sites'] = self._precursor_df.mod_sites.astype('U')

        self.sort_by_nAA()

    def predict_rt(self):
        """ add 'rt_pred' and 'irt_pred' into columns """
        self._precursor_df = self.model_manager.rt_model.predict(
            self._precursor_df, verbose=self.verbose
        )
        self.model_manager.rt_model.rt_to_irt_pred(self._precursor_df)

    def predict_mobility(self):
        if 'precursor_mz' not in self._precursor_df.columns:
            self.calc_precursor_mz()
        """ add 'ccs_pred' and 'mobility_pred' into columns """
        self._precursor_df = self.model_manager.ccs_model.predict(
            self._precursor_df, verbose=self.verbose
        )
        self.model_manager.ccs_model.ccs_to_mobility_pred(
            self._precursor_df
        )

    def load_fragment_intensity_df(self, **kwargs):
        self.predict_fragment_intensity_df(**kwargs)

    def predict_fragment_intensity_df(self, **kwargs):
        frag_inten_df = self.model_manager.ms2_model.predict(
            self._precursor_df,
            verbose=self.verbose,
        )

        charged_frag_list = []
        for frag_type in self._fragment_mz_df.columns.values:
            if frag_type in frag_inten_df:
                charged_frag_list.append(frag_type)
        self._fragment_mz_df = self._fragment_mz_df[
            charged_frag_list
        ]
        self._fragment_intensity_df = frag_inten_df[
            charged_frag_list
        ]*self.intensity_factor
        self._fragment_intensity_df[self._fragment_mz_df==0] = 0
        